In [10]:
#importing libraries
import pandas as pd
import numpy as np
import string  
import re
# setting the viewing limit for dataframe
pd.set_option("display.max_rows", 101)

from bs4 import BeautifulSoup
import json
import requests

In [11]:
actorProducerDirector = pd.DataFrame(columns= ['Name', 'Movie Name', 'MovieId', 'Year','Role'])

In [21]:
crew = pd.read_csv('data/pictures_combined2.csv')

In [26]:
# getting list of name_ids for each crew member.

crew_list = crew[['director_id','Director']]
crew_df = pd.DataFrame(crew_list).rename(columns={'director_id':'imdb_name_id','Director':'name'})
crew_df = crew_df.drop_duplicates('imdb_name_id')
crew_df = crew_df.reset_index()
crew_df

,index,imdb_name_id,name
0,0,nm0587277,Lewis Milestone
1,1,nm0097648,Frank Borzage
2,2,nm0920074,William A. Wellman
3,3,nm0922327,Roland West
4,4,nm0191899,Irving Cummings
...,...,...,...
286,554,nm0003506,James Mangold
287,556,nm0169806,Taika Waititi
288,557,nm0680846,Todd Phillips
289,559,nm0000876,Noah Baumbach


In [27]:
def fillPerRole(credits, name, typeOfRole):
    role = '';
    if typeOfRole == 'actor':
        role = 'Actor'
    elif typeOfRole == 'director':
        role = 'Director'
    elif typeOfRole == 'producer':
        role = 'Producer'
    
    for credit in credits:
        movieName = credit.find('a', href=True).text
        # HERE I NEED TO ADD SOMETHING TO SKIP TV AND SHORT AND DOCUMENTARIES
        movieId = credit['id'].split('-',1)[1]
        year = credit.find('span', attrs={'class': 'year_column'}).text.replace('\n', '')
        new_row=len(actorProducerDirector)+1
        actorProducerDirector.loc[new_row,['Name']] = name
        actorProducerDirector.loc[new_row,['Movie Name']] = movieName
        actorProducerDirector.loc[new_row,['MovieId']] = movieId
        actorProducerDirector.loc[new_row,['Year']] = year
        actorProducerDirector.loc[new_row,['Role']] = role


In [28]:
def fillActorProducerDirector(nameid):
    header = {'Accept-Language':'en-US;q=0.7, es-MX;q=0.3'}
    url = "https://www.imdb.com/name/"+nameid+"/"
    req = requests.get(url, headers=header).text
    soup = BeautifulSoup(req)
    name = soup.find('div', attrs={'id': 'name-overview-widget'}).find('span', attrs={'class': 'itemprop'}).text

    filmography = soup.findAll('div', attrs={'class': 'filmo-category-section'})
    for film in filmography:
        actorCredits = film.findAll('div', id=lambda value: value and value.startswith('actor'))
        directorCredits = film.findAll('div', id=lambda value: value and value.startswith('director'))
        producerCredits = film.findAll('div', id=lambda value: value and value.startswith('producer'))
        fillPerRole(actorCredits, name, 'actor')
        fillPerRole(directorCredits, name,  'director')
        fillPerRole(producerCredits, name, 'producer')

In [29]:
for i, nameid in enumerate(crew_df['imdb_name_id']):
    name = crew_df.at[i, 'name']    
    url = "https://www.imdb.com/name/"+nameid+"/"
    req =requests.get(url).text
    soup=BeautifulSoup(req, 'lxml')
    print(i)
    fillActorProducerDirector(nameid)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [30]:
actorProducerDirector

,Name,Movie Name,MovieId,Year,Role
1,Lewis Milestone,Arrest and Trial,tt0056737,1963-1964,Director
2,Lewis Milestone,The Richard Boone Show,tt0056783,1964,Director
3,Lewis Milestone,Mutiny on the Bounty,tt0056264,1962,Director
4,Lewis Milestone,Ocean's 11,tt0054135,1960,Director
5,Lewis Milestone,Pork Chop Hill,tt0053183,1959,Director
...,...,...,...,...,...
21018,Bong Joon Ho,Sea Fog,tt3303728,2014,Producer
21019,Bong Joon Ho,Doomsday Book,tt2297164,2012,Actor
21020,Bong Joon Ho,Crush and Blush,tt1173949,2008,Actor
21021,Bong Joon Ho,No Blood No Tears,tt0310000,2002,Actor


In [332]:
actorProducerDirector.to_csv('temp/temp_actorProducerDirector.csv',index=False)

In [333]:
directors = actorProducerDirector.loc[actorProducerDirector.Role == 'Director']

In [334]:
directors

,Name,Movie Name,MovieId,Year,Role
1,Lewis Milestone,Arrest and Trial,tt0056737,1963-1964,Director
2,Lewis Milestone,The Richard Boone Show,tt0056783,1964,Director
3,Lewis Milestone,Mutiny on the Bounty,tt0056264,1962,Director
4,Lewis Milestone,Ocean's 11,tt0054135,1960,Director
5,Lewis Milestone,Pork Chop Hill,tt0053183,1959,Director
...,...,...,...,...,...
21010,Bong Joon Ho,Memories of Murder,tt0353969,2003,Director
21011,Bong Joon Ho,Barking Dogs Never Bite,tt0269743,2000,Director
21012,Bong Joon Ho,Incoherence,tt1997399,1994,Director
21013,Bong Joon Ho,Memories in My Frame,tt2999786,1994,Director


In [335]:
directors1 = pd.merge(directors,crew_df, how='left', left_on='Name',right_on='name' )

In [336]:
directors1 = directors1.drop(columns=['index','name'])

In [337]:
directors1 = directors1.rename(columns={'Role':'Director'})

In [338]:
directproducers = actorProducerDirector.loc[actorProducerDirector.Role == 'Producer']

In [339]:
directors1.loc[directors1.Name=='Lewis Milestone']

,Name,Movie Name,MovieId,Year,Director,imdb_name_id
0,Lewis Milestone,Arrest and Trial,tt0056737,1963-1964,Director,nm0587277
1,Lewis Milestone,The Richard Boone Show,tt0056783,1964,Director,nm0587277
2,Lewis Milestone,Mutiny on the Bounty,tt0056264,1962,Director,nm0587277
3,Lewis Milestone,Ocean's 11,tt0054135,1960,Director,nm0587277
4,Lewis Milestone,Pork Chop Hill,tt0053183,1959,Director,nm0587277
...,...,...,...,...,...,...
624,Lewis Milestone,Seven Sinners,tt0016334,1925,Director,nm0587277
625,Lewis Milestone,Fit to Win,tt0010131,1919,Director,nm0587277
626,Lewis Milestone,Positive,tt0313492,1918,Director,nm0587277
627,Lewis Milestone,Posture,tt0304577,1918,Director,nm0587277


In [340]:
directproducers = directproducers.rename(columns={'Role':'Producer'})
directproducers.Producer = directproducers.Producer.str.replace('Producer','Yes')

In [341]:
directors2 = pd.merge(directors1, directproducers, how='left', on =['Name','Movie Name','MovieId','Year'])

In [342]:
len(directors2)

11567

In [343]:
#dropping duplicated entries

directors2 = directors2.drop_duplicates(subset=['MovieId'])
len(directors2)

10392

In [346]:
directors2 = directors2.fillna('No')

In [347]:
directors2.Year = directors2.Year.str.replace('\xa0', '')

In [348]:
directors2.Year = directors2.Year.str.rstrip('/')
directors2.Year = directors2.Year.str.rstrip('-')

In [349]:
directors2[['Year1','Year2']] = directors2.Year.str.split('-', expand = True)

In [350]:
directors2[['Year3','Year4']] = directors2.Year1.str.split('/', expand = True)

In [351]:
directors2 = directors2.drop(columns=['Year','Year1','Year2','Year4'])

In [352]:
directors2.Year3 = directors2.Year3.str.strip() 

In [353]:
directors2 = directors2.loc[directors2.Year3 != '']

In [354]:
directors2.Year3 = directors2.Year3.astype(int)

In [355]:
directors2 = directors2.rename(columns={'Year3':'Year'})

In [356]:
# spliting directors into ones before 1927 and after, so that we can count how many projects they had.
directors_early = directors2.loc[directors2['Year'] <=1927]
directors_early = directors_early.groupby(['Name','imdb_name_id'])['Count'].sum()
directors_early =directors_early.reset_index()

directors_early['Year'] = 1927

In [357]:
directors_o = directors2.loc[directors2['Year'] >=1928]
directors_o = directors_o.groupby(['Name','imdb_name_id','Year'])['Count'].sum()
directors_o = directors_o.reset_index()

In [358]:
#combining both dataframes
directors_all = directors_early.append(directors_o, ignore_index=True)
directors_all

,Name,imdb_name_id,Count,Year
0,Alexander Hall,nm0355284,5,1927
1,Alexander Korda,nm0466099,41,1927
2,Alfred E. Green,nm0337586,38,1927
3,Alfred Hitchcock,nm0000033,8,1927
4,Cecil B. DeMille,nm0001124,59,1927
...,...,...,...,...
5570,Yorgos Lanthimos,nm0487166,1,2015
5571,Yorgos Lanthimos,nm0487166,1,2017
5572,Yorgos Lanthimos,nm0487166,1,2018
5573,Yorgos Lanthimos,nm0487166,1,2019


In [359]:
directors_all_f = directors_all.set_index(["Name",'imdb_name_id','Year']) 
directors_all_f = directors_all_f.groupby(level=[0]).cumsum()
directors_all_f = directors_all_f.reset_index()
directors_all_f = directors_all_f.groupby(['Name','imdb_name_id','Year'])['Count'].max()
directors_all_f = directors_all_f.reset_index()
directors_all_f
directors_all_f = directors_all_f.rename(columns = {'Name':'Dir_Name','Count':'Dir_Cum_Films'})

In [360]:
directors_all_f.columns, crew.columns

(Index(['Dir_Name', 'imdb_name_id', 'Year', 'Dir_Cum_Films'], dtype='object'),
 Index(['year_film', 'year_ceremony', 'ceremony', 'category', 'name', 'film',
        'winner', 'imdb_title_id', 'title', 'year', 'genre', 'duration',
        'country', 'language', 'director', 'writer', 'production_company',
        'actors', 'description', 'first_director', 'second_director',
        'Director', 'director_id'],
       dtype='object'))

In [361]:
pictures_combined3 = pd.merge(crew,directors_all_f, 
                              how='left', 
                              left_on=['director_id','year_film'],
                              right_on=['imdb_name_id','Year'])

In [362]:
pictures_combined3.isna().sum()
pictures_combined3.loc[pictures_combined3.Dir_Cum_Films.isna()]
#8 rows were not merged. They will be added manually

,year_film,year_ceremony,ceremony,category,name,film,winner,imdb_title_id,title,year,...,actors,description,first_director,second_director,Director,director_id,Dir_Name,imdb_name_id,Year,Dir_Cum_Films
91,1939,1940,12,OUTSTANDING PRODUCTION,Selznick International Pictures,GONE WITH THE WIND,True,tt0031381,GONE WITH THE WIND,1939.0,...,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh,...",A manipulative woman and a roguish man conduct...,Victor Fleming,George Cukor,Victor Fleming,nm0281808,NaN,NaN,NaN,NaN
92,1939,1940,12,OUTSTANDING PRODUCTION,Metro-Goldwyn-Mayer,"GOODBYE, MR. CHIPS",False,tt0031385,"GOODBYE, MR. CHIPS",1939.0,...,"Robert Donat, Greer Garson, Terry Kilburn, Joh...",An aged teacher and former headmaster of a boa...,Sam Wood,Sidney Franklin,Sam Wood,nm0939992,NaN,NaN,NaN,NaN
98,1939,1940,12,OUTSTANDING PRODUCTION,Metro-Goldwyn-Mayer,THE WIZARD OF OZ,False,tt0032138,THE WIZARD OF OZ,1939.0,...,"Judy Garland, Frank Morgan, Ray Bolger, Bert L...",Dorothy Gale is swept away from a farm in Kans...,Victor Fleming,George Cukor,Victor Fleming,nm0281808,NaN,NaN,NaN,NaN
270,1970,1971,43,BEST PICTURE,"Ross Hunter, Producer",AIRPORT,False,tt0065377,AIRPORT,1970.0,...,"Burt Lancaster, Dean Martin, Jean Seberg, Jacq...","A bomber on board an airplane, an airport almo...",George Seaton,Henry Hathaway,George Seaton,nm0780833,NaN,NaN,NaN,NaN
458,2007,2008,80,BEST PICTURE,"Scott Rudin, Ethan Coen and Joel Coen, Producers",NO COUNTRY FOR OLD MEN,True,tt0477348,NO COUNTRY FOR OLD MEN,2007.0,...,"Tommy Lee Jones, Javier Bardem, Josh Brolin, W...",Violence and mayhem ensue after a hunter stumb...,Ethan Coen,Joel Coen,Ethan Coen,nm0001053,NaN,NaN,NaN,NaN
469,2009,2010,82,BEST PICTURE,"Kathryn Bigelow, Mark Boal, Nicolas Chartier a...",THE HURT LOCKER,True,tt0887912,THE HURT LOCKER,2009.0,...,"Jeremy Renner, Anthony Mackie, Brian Geraghty,...","During the Iraq War, a Sergeant recently assig...",Kathryn Bigelow,NaN,Kathryn Bigelow,nm0000941,NaN,NaN,NaN,NaN
472,2009,2010,82,BEST PICTURE,"Joel Coen and Ethan Coen, Producers",A SERIOUS MAN,False,tt1019452,A SERIOUS MAN,2009.0,...,"Michael Stuhlbarg, Richard Kind, Fred Melamed,...","Larry Gopnik, a Midwestern physics teacher, wa...",Ethan Coen,Joel Coen,Ethan Coen,nm0001053,NaN,NaN,NaN,NaN
483,2010,2011,83,BEST PICTURE,"Scott Rudin, Ethan Coen and Joel Coen, Producers",TRUE GRIT,False,tt1403865,TRUE GRIT,2010.0,...,"Jeff Bridges, Hailee Steinfeld, Matt Damon, Jo...",A stubborn teenager enlists the help of a toug...,Ethan Coen,Joel Coen,Ethan Coen,nm0001053,NaN,NaN,NaN,NaN


In [363]:
pictures_combined3.at[91,'Dir_Cum_Films'] = 42
pictures_combined3.at[92,'Dir_Cum_Films'] = 68
pictures_combined3.at[98,'Dir_Cum_Films'] = 42
pictures_combined3.at[270,'Dir_Cum_Films'] = 34
pictures_combined3.at[458,'Dir_Cum_Films'] = 18
pictures_combined3.at[472,'Dir_Cum_Films'] = 20
pictures_combined3.at[483,'Dir_Cum_Films'] = 21
pictures_combined3.at[469,'Dir_Cum_Films'] = 14


In [364]:
pictures_combined3.isna().sum()
pictures_combined3 = pictures_combined3.drop(columns=['Dir_Name','imdb_name_id','Year','director'])

In [365]:
len(pictures_combined3)

563

In [368]:
directproducers.columns
directproducers = directproducers.rename(columns={'MovieId':'imdb_title_id'})
directproducers1 = directproducers[['imdb_title_id','Producer']]
directproducers1 = directproducers1.drop_duplicates(subset=['imdb_title_id'])


In [369]:
#directproducers1.to_csv('temp/directtprodu.csv')

In [370]:
len(directproducers1)

4630

In [371]:
pictures_combined3 = pd.merge(pictures_combined3, directproducers1,
                              how='left',
                             on='imdb_title_id'
                             )

In [372]:
pictures_combined3

,year_film,year_ceremony,ceremony,category,name,film,winner,imdb_title_id,title,year,...,writer,production_company,actors,description,first_director,second_director,Director,director_id,Dir_Cum_Films,Producer
0,1928,1928,1,OUTSTANDING PICTURE,The Caddo Company,THE RACKET,False,tt0019304,THE RACKET,1928.0,...,"Bartlett Cormack, Bartlett Cormack",The Caddo Company,"Thomas Meighan, Louis Wolheim, Marie Prevost, ...",An honest police captain vows to bring down a ...,Lewis Milestone,NaN,Lewis Milestone,nm0587277,13.0,NaN
1,1927,1928,1,OUTSTANDING PICTURE,Fox,7TH HEAVEN,False,tt0018379,7TH HEAVEN,1927.0,...,"Austin Strong, Benjamin Glazer",Frank Borzage Production,"Janet Gaynor, Charles Farrell, Albert Gran, Da...","A street cleaner saves a young woman's life, a...",Frank Borzage,NaN,Frank Borzage,nm0097648,58.0,NaN
2,1927,1928,1,OUTSTANDING PICTURE,Paramount Famous Lasky,WINGS,True,tt0018578,WINGS,1927.0,...,"John Monk Saunders, Hope Loring",Paramount Famous Lasky Corporation,"Clara Bow, Charles 'Buddy' Rogers, Richard Arl...","Two young men, one rich, one middle class, who...",William A. Wellman,Harry d'Abbadie d'Arrast,William A. Wellman,nm0920074,12.0,NaN
3,1929,1929,2,OUTSTANDING PICTURE,Feature Productions,ALIBI,False,tt0019630,ALIBI,1929.0,...,"Roland West, C. Gardner Sullivan",Feature Productions,"Chester Morris, Harry Stubbs, Mae Busch, Elean...","Chick Williams, a prohibition gangster, rejoin...",Roland West,NaN,Roland West,nm0922327,12.0,Yes
4,1928,1929,2,OUTSTANDING PICTURE,Fox,IN OLD ARIZONA,False,tt0020018,IN OLD ARIZONA,1928.0,...,"O. Henry, Tom Barry",Fox Film Corporation,"Warner Baxter, Edmund Lowe, Dorothy Burgess","A charming, happy-go-lucky bandit in old Arizo...",Irving Cummings,NaN,Irving Cummings,nm0191899,41.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,2019,2020,92,BEST PICTURE,"Amy Pascal, Producer",LITTLE WOMEN,False,tt3281548,LITTLE WOMEN,2019.0,...,"Louisa May Alcott, Clare Niederpruem",Main Dog Productions,"'Saoirse Ronan', 'Emma Watson', 'Florence Pugh...",A modern retelling of Louisa May Alcott's clas...,Greta Gerwig,NaN,Greta Gerwig,nm1950086,3.0,NaN
559,2019,2020,92,BEST PICTURE,"Noah Baumbach and David Heyman, Producers",MARRIAGE STORY,False,tt7653254,MARRIAGE STORY,2019.0,...,Noah Baumbach,Heyday Films,"Scarlett Johansson, Merritt Wever, Adam Driver...",Noah Baumbach's incisive and compassionate loo...,Noah Baumbach,NaN,Noah Baumbach,nm0000876,13.0,Yes
560,2019,2020,92,BEST PICTURE,"Sam Mendes, Pippa Harris, Jayne-Ann Tenggren a...",1917;,False,tt8579674,NaN,NaN,...,NaN,NaN,NaN,NaN,Sam Mendes,NaN,Sam Mendes,nm0005222,12.0,Yes
561,2019,2020,92,BEST PICTURE,"David Heyman, Shannon McIntosh and Quentin Tar...",ONCE UPON A TIME… IN HOLLYWOOD,False,tt7131622,NaN,NaN,...,NaN,NaN,NaN,NaN,Quentin Tarantino,NaN,Quentin Tarantino,nm0000233,20.0,Yes


In [377]:
pictures_combined3.Producer = pictures_combined3.Producer.fillna('No')

In [379]:
pictures_combined3.to_csv('data/pictures_combined3.csv',index=False)